<a target="_blank" href="https://colab.research.google.com/github/nkrbrock/qa-alpaca-fine-tuning/blob/master/notebooks/qa_alpaca_fine_tune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Setup

In [ ]:
!pip install torch transformers sentencepiece accelerate bitsandbytes python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    GenerationConfig, LlamaTokenizer, LlamaForCausalLM, 
    Trainer, TrainingArguments
)
from huggingface_hub import notebook_login
from Levenshtein import distance

# Preprocess

In [ ]:
dataset = '/content/qac_data.csv'
data = pd.read_csv(dataset, encoding='utf-8')

In [ ]:
data.head()
entries = data['0'].to_list()

In [ ]:
train = int(len(entries) * .8)
test = train + int(len(entries) * .1)

In [ ]:
training = entries[:train]
testing = entries[train:test]
evaluating = entries[test:]

In [ ]:
print(len(training))
print(len(testing))
print(len(evaluating))

14085
1760
1762


In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Using pad_token, but it is not set yet.


In [ ]:
model = LlamaForCausalLM.from_pretrained(
    "chainyo/alpaca-lora-7b",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/39 [00:00<?, ?it/s]

# Process Data

In [ ]:
def preprocess_data(dataset):
    tokenized_texts = []
    
    for example in dataset:
        text = example
        tokenized_text = tokenizer.encode(text, return_tensors="pt")
        tokenized_texts.append(tokenized_text)

    return tokenized_texts

In [ ]:
train_text = preprocess_data(training)
val_text = preprocess_data(testing)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx]

In [ ]:
train_dataset = CustomDataset(train_text)
val_dataset = CustomDataset(val_text)

In [ ]:
train_dataset[0][0]

tensor([    0, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,  5618,
         4234,   947,   278, 24834,  3780,  1065,  3412, 29973,    13,    13,
         2277, 29937, 10567, 29901,    13,  1576,  1024,   310,   278,  4654,
         8580,   338, 24834,  3780, 29936,   372,  6057,  3412,   278,  9755,
         2625,   310,  4007, 29891,  2849, 29889,   450, 11582,  8580,   338,
          278, 16430, 24588,  1078, 29889,    13,    13,  2277, 29937, 13291,
        29901,    13,  7900, 29891,  2849])

In [ ]:
def create_dict(dataset):
  dicts = []

  for data in dataset:
    info = {}
    indexes = data[0].tolist()
    info['input_ids'] = indexes
    info['attention_mask'] = [1] * len(indexes)
    info['labels'] = indexes

    dicts.append(info)

  return dicts

In [ ]:
train_values = create_dict(train_dataset)
val_values = create_dict(val_dataset)

In [ ]:
train_values[0]

{'input_ids': [0,
  13866,
  338,
  385,
  15278,
  393,
  16612,
  263,
  3414,
  29889,
  14350,
  263,
  2933,
  393,
  7128,
  2486,
  1614,
  2167,
  278,
  2009,
  29889,
  13,
  13,
  2277,
  29937,
  2799,
  4080,
  29901,
  13,
  5618,
  4234,
  947,
  278,
  24834,
  3780,
  1065,
  3412,
  29973,
  13,
  13,
  2277,
  29937,
  10567,
  29901,
  13,
  1576,
  1024,
  310,
  278,
  4654,
  8580,
  338,
  24834,
  3780,
  29936,
  372,
  6057,
  3412,
  278,
  9755,
  2625,
  310,
  4007,
  29891,
  2849,
  29889,
  450,
  11582,
  8580,
  338,
  278,
  16430,
  24588,
  1078,
  29889,
  13,
  13,
  2277,
  29937,
  13291,
  29901,
  13,
  7900,
  29891,
  2849],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

# Train & Upload Model

In [ ]:
notebook_login()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_values[:100],
    eval_dataset=val_values[:20],
    tokenizer=tokenizer,
)

/content/./results is already a clone of https://huggingface.co/nathan-brock/results. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=300, training_loss=0.0, metrics={'train_runtime': 302.1659, 'train_samples_per_second': 0.993, 'train_steps_per_second': 0.993, 'total_flos': 1086207646408704.0, 'train_loss': 0.0, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3650 in push_to_hub              │
│                                                                                                  │
│   3647 │   │                                                                                     │
│   3648 │   │   # Needs to be executed on all processes for TPU training, but will only save on   │
│   3649 │   │   # self.args.should_save.                                                          │
│ ❱ 3650 │   │   self.save_model(_internal_call=True)                                              │
│   3651 │   │                                                                                     │
│   3652 │   │   # Only push from one node.                                                        │
│   3653 │   │   if not self.is_world_process_zero():                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2830 in save_model               │
│                                                                                                  │
│   2827 │   │   │   │   │   self.deepspeed.save_checkpoint(output_dir)                            │
│   2828 │   │                                                                                     │
│   2829 │   │   elif self.args.should_save:                                                       │
│ ❱ 2830 │   │   │   self._save(output_dir)                                                        │
│   2831 │   │                                                                                     │
│   2832 │   │   # Push to the Hub when `save_model` is called by the user.                        │
│   2833 │   │   if self.args.push_to_hub and not _internal_call:                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2886 in _save                    │
│                                                                                                  │
│   2883 │   │   │   │   else:                                                                     │
│   2884 │   │   │   │   │   torch.save(state_dict, os.path.join(output_dir, WEIGHTS_NAME))        │
│   2885 │   │   else:                                                                             │
│ ❱ 2886 │   │   │   self.model.save_pretrained(                                                   │
│   2887 │   │   │   │   output_dir, state_dict=state_dict, safe_serialization=self.args.save_saf  │
│   2888 │   │   │   )                                                                             │
│   2889                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1762 in save_pretrained   │
│                                                                                                  │
│   1759 │   │                                                                                     │
│   1760 │   │   # Save the model                                                                  │
│   1761 │   │   if state_dict is None:                                                            │
│ ❱ 1762 │   │   │   state_dict = model_to_save.state_dict()                                       │
│   1763 │   │                                               

#Evaluate

In [ ]:
evaluating[0]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat city did you build through bloody crimes?\n\n### Input:\nI said, “Listen, you leaders of Jacob, you rulers of the nation of Israel! You ought to know what is just, yet you hate what is good, and love what is evil. You flay my people’s skin and rip the flesh from their bones. You devour my people’s flesh, strip off their skin, and crush their bones. You chop them up like flesh in a pot – like meat in a kettle. Someday these sinners will cry to the Lord for help, but he will not answer them. He will hide his face from them at that time, because they have done such wicked deeds.” This is what the Lord says: “The prophets who mislead my people are as good as dead. If someone gives them enough to eat, they offer an oracle of peace. But if someone does not give them food, they are ready to declare war on him. Therefore night will fall, and you will receive no v

In [ ]:
prompts = []
answers = []

for i in evaluating:
  prompt_answer = i.rsplit('\n', 1)
  prompts.append(prompt_answer[0])
  answers.append(prompt_answer[1])

In [ ]:
answers[0]

'Zion'

In [ ]:
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
)

model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)

In [ ]:
predictions = []

for i in prompts:
  input_ids = tokenizer(i, return_tensors="pt").input_ids
  input_ids = input_ids.to(model.device)

  with torch.no_grad():
      outputs = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
      )

  response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
  prediction = response.rsplit('\n', 1)
  predictions.append(prediction[1])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:5                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'function' object has no attribute 'device'

In [ ]:
predictions[:5]

In [ ]:
scores = []

for i in range(len(predictions)):
    sim = 1 - (distance(predictions[i], answers[i])/max(len(predictions[i]), len(answers[i])))
    scores.append(sim)

In [ ]:
threshold = 0.8
high_scores = sum(score >= threshold for score in scores)
print(f'Number of responses that are at least 80% similar: {high_scores}')